# ML with PySpark

The primary machine learning API is contained in `pyspark.ml`. It is based on PySpark DataFrames. The well-known RDD-based API `pyspark.mllib` is mainly provided for legacy purposes and might become deprecated at some point.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import (
    StringIndexer,
    OneHotEncoder,
    VectorAssembler
)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from utils import download_textfile

## Start session

In [ ]:
# start session
spark = SparkSession.builder \
    .appName('Logistic Regression') \
    .getOrCreate()

## Import data

In [ ]:
# download CSV file
url = 'https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv'

download_textfile(url, save_path='../data/titanic.csv')

In [ ]:
# import CSV file
df = spark.read.csv('../data/titanic.csv', header=True, inferSchema=True)

print(f'The data has {df.count()} rows and {len(df.columns)} columns')

In [ ]:
# show summary
df.show(10)
df.printSchema()

In [ ]:
# select columns
data_columns = [
    'Survived',
    'Pclass',
    'Sex',
    'Age',
    'Siblings/Spouses Aboard',
    'Parents/Children Aboard',
    'Fare'
]

df = df.select(data_columns) # df = df.drop('Name')

In [ ]:
# drop NaNs
df = df.na.drop() # note that there are no NaNs

# rename column
df = df.withColumnRenamed('Survived', 'label')

## Create pipeline

In [ ]:
# create transformations
sex_indexer = StringIndexer(inputCol='Sex', outputCol='SexIdx')
sex_encoder = OneHotEncoder(inputCol='SexIdx', outputCol='SexVec')

feature_columns = [
    'Pclass',
    'SexVec',
    'Age',
    'Siblings/Spouses Aboard',
    'Parents/Children Aboard',
    'Fare'
]

assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

In [ ]:
# create model
lr = LogisticRegression(
    featuresCol='features',
    labelCol='label',
    maxIter=10,
    regParam=0.1,
)

In [ ]:
# create pipeline
stages = [
    sex_indexer, # estimator
    sex_encoder, # estimator
    assembler, # transformer
    lr # estimator
]

pipeline = Pipeline(stages=stages) # contains transformers and estimators

## Fit model

In [ ]:
# split data
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# fit model
model = pipeline.fit(train_df)

In [ ]:
# get summary
train_summary = model.stages[-1].summary

train_acc = train_summary.accuracy
train_roc_auc = train_summary.areaUnderROC

train_pr = train_summary.pr.toPandas()
train_roc = train_summary.roc.toPandas()

print(f'Train acc.: {train_acc}')

In [ ]:
# plot PR curve
fig, ax = plt.subplots(figsize=(5, 3.5))
ax.plot(train_pr['recall'], train_pr['precision'])
ax.set_title('Train PR curve')
ax.set(xlabel='recall', ylabel='precision')
ax.set(xlim=(0, 1), ylim=(0, 1))
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

In [ ]:
# plot ROC curve
fig, ax = plt.subplots(figsize=(5, 3.5))
ax.plot(train_roc['FPR'], train_roc['TPR'])
ax.set_title('Train ROC curve')
ax.set(xlabel='FPR', ylabel='TPR')
ax.set(xlim=(0, 1), ylim=(0, 1))
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

## Test model

In [ ]:
# make test predictions
pred_df = model.transform(test_df)

In [ ]:
# show summary
pred_df.show(10)
pred_df.printSchema()

In [ ]:
# count TP, FP, TN, FN
tp = pred_df.filter('label = 1 AND prediction = 1').count()
fp = pred_df.filter('label = 0 AND prediction = 1').count()
tn = pred_df.filter('label = 0 AND prediction = 0').count()
fn = pred_df.filter('label = 1 AND prediction = 0').count()

In [ ]:
# calculate accuracy
test_acc = (tp + tn) / (tp + fp + tn + fn)

print(f'Test acc.: {test_acc:.4f}')

In [ ]:
# assemble confusion matrix
y_pred = pred_df.select('prediction').toPandas().to_numpy()
y_true = pred_df.select('label').toPandas().to_numpy()

confmat = confusion_matrix(y_true, y_pred)

In [ ]:
# plot confusion matrix
fig, ax = plt.subplots(figsize=(5, 3.5))
disp = ConfusionMatrixDisplay(confusion_matrix=confmat, display_labels=(0, 1))
disp.plot(ax=ax)
fig.tight_layout()

In [ ]:
# instantiate evaluators
roc_auc_metric = BinaryClassificationEvaluator(
    rawPredictionCol='rawPrediction',
    labelCol='label',
    metricName='areaUnderROC'
)

pr_auc_metric = BinaryClassificationEvaluator(
    rawPredictionCol='rawPrediction',
    labelCol='label',
    metricName='areaUnderPR'
)

In [ ]:
# calculate PR and ROC AUC
pr_auc = pr_auc_metric.evaluate(pred_df)
roc_auc = roc_auc_metric.evaluate(pred_df)

print(f'Test PR AUC: {pr_auc:.4f}')
print(f'Test ROC AUC: {roc_auc:.4f}')

## Stop session

In [ ]:
# stop session
spark.stop()